In [1]:
import yfinance as yf
import pandas as pd
from ta.momentum import RSIIndicator
from ta.trend import MACD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [2]:
ticker = "RELIANCE.NS"

data = yf.download(ticker, period="1y", auto_adjust=True)
data.head()


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2024-06-26,1514.025024,1518.974976,1445.125000,1446.050049,22014882
2024-06-27,1530.550049,1537.500000,1506.000000,1513.750000,28383030
2024-06-28,1565.400024,1581.000000,1531.025024,1531.025024,28957336
2024-07-01,1560.150024,1579.400024,1555.675049,1562.525024,7724306
2024-07-02,1565.175049,1573.500000,1557.000000,1569.500000,7749246


In [3]:
# Step 3: Clean and Prepare 'Close' and 'Volume'
close_series = pd.Series(data['Close'].to_numpy().flatten(), index=data.index)
volume_series = pd.Series(data['Volume'].to_numpy().flatten(), index=data.index)

df = pd.DataFrame({
    'Close': close_series,
    'Volume': volume_series
})

df.head()


,Close,Volume
Date,,
2024-06-26,1514.025024,22014882
2024-06-27,1530.550049,28383030
2024-06-28,1565.400024,28957336
2024-07-01,1560.150024,7724306
2024-07-02,1565.175049,7749246


In [4]:
# RSI (Relative Strength Index)
df['RSI'] = RSIIndicator(close=df['Close'], window=14).rsi()

# MACD (Moving Average Convergence Divergence)
macd = MACD(close=df['Close'])
df['MACD'] = macd.macd()
df['MACD_Signal'] = macd.macd_signal()


In [5]:
# Target: 1 if tomorrow's price is higher, else 0
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)


In [6]:
df.dropna(inplace=True)
df.head()


,Close,Volume,RSI,MACD,MACD_Signal,Target
Date,,,,,,
2024-08-13,1463.625000,6265064,39.395215,-21.949765,-18.902003,0
2024-08-14,1461.849976,6267466,39.010049,-21.679008,-19.457404,1
2024-08-16,1478.199951,9416902,44.402171,-19.915550,-19.549033,1
2024-08-19,1488.400024,13797742,47.519417,-17.493285,-19.137884,1
2024-08-20,1495.949951,8411808,49.764577,-14.793872,-18.269081,1


In [7]:
X = df[['RSI', 'MACD', 'Volume']]
y = df['Target']


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False
)


In [9]:
model = LogisticRegression()
model.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [10]:
y_pred = model.predict(X_test)

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['Down', 'Up'])
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.4318

Classification Report:
               precision    recall  f1-score   support

        Down       0.43      1.00      0.60        19
          Up       0.00      0.00      0.00        25

    accuracy                           0.43        44
   macro avg       0.22      0.50      0.30        44
weighted avg       0.19      0.43      0.26        44

Confusion Matrix:
 [[19  0]
 [25  0]]


C:\Users\Rjkin\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Rjkin\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Rjkin\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capital

In [11]:
pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred
}).reset_index(drop=True).head(15)


,Actual,Predicted
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
5,0,0
6,0,0
7,1,0
8,0,0
9,1,0


In [12]:
df['Target'].value_counts()


Target
1    111
0    106
Name: count, dtype: int64

In [13]:
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[19  0]
 [25  0]]


In [14]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()
def send_telegram_alert(message: str):
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message
    }

    response = requests.post(url, data=payload)
    if response.status_code == 200:
        print("✅ Telegram alert sent successfully!")
    else:
        print(f"❌ Failed to send alert: {response.text}")


In [15]:
send_telegram_alert("Hello, this is a test message!")

✅ Telegram alert sent successfully!


In [16]:
if y_pred[-1] == 1:
    send_telegram_alert(f"📈 ML predicts RELIANCE.NS will go UP tomorrow!\n accuracy = {accuracy:.4f}")
else:
    send_telegram_alert(f"📉 ML predicts RELIANCE.NS will go DOWN tomorrow!\n accuracy = {accuracy:.4f}")


✅ Telegram alert sent successfully!
